Problem Statement:
News headlines cover diverse topics, and manually categorizing them is time-consuming and error-prone. An automated system is needed to accurately classify news headlines into predefined categories.

Objective:
To fine-tune a BERT-based transformer model to classify news headlines from the AG News dataset into topic categories and evaluate its performance using accuracy and F1-score.

In [27]:
#Step 1: Setup & Install Dependencies
!pip install datasets transformers torch scikit-learn gradio accelerate

Defaulting to user installation because normal site-packages is not writeable
  Using cached gradio-6.5.1-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached fastapi-0.128.0-py3-none-any.whl.metadata (30 kB)
  Using cached ffmpy-1.0.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached gradio_client-2.0.3-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.22-py3-none-any.whl.metadata (1.8 kB)
  Using cached safehttpx-0.1.7-py3-none-any.whl.metadata (4.2 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached starlette-0.52.1-py3-none-any.whl.metadata (6.3 kB)
  Using cached tomlkit-0.13.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached typer-0.21.1-py3-none-any.whl.metadata (16 kB)
  Using cached uvicorn-0.40.0-py3-none-any.whl.metadata 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))


CUDA available: True
GPU: NVIDIA RTX A4000


In [3]:
import datasets
import transformers
import torch
import sklearn
import gradio

print("All packages installed correctly!")


C:\Users\fast\.conda\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All packages installed correctly!


Step 2: Load the AG News Dataset

AG News has 4 classes:

0 → World

1 → Sports

2 → Business

3 → Sci/Tech

In [5]:
from datasets import load_dataset

dataset = load_dataset("ag_news")

print(dataset)


C:\Users\fast\.conda\envs\myenv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\fast\.cache\huggingface\hub\datasets--ag_news. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For bette

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


Step 3: Tokenization & Preprocessing

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")


C:\Users\fast\.conda\envs\myenv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\fast\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map: 100%|████████████████████████████████████████████████████████████████| 7600/7600 [00:07<00:00, 1004.35 examples/s]


Step 4: Load BERT for Sequence Classification

In [9]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=4
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
Step 5: Training Configuration

In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,  # GPU acceleration
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)


Step 6: Evaluation Metrics (Accuracy & F1)

In [15]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted")
    }


In [ ]:
Step 7: Fine-Tune the Model

In [17]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


C:\Users\fast\AppData\Local\Temp\ipykernel_17240\2441128132.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.198300,0.175362,0.945263,0.945298
2,0.130500,0.178245,0.950395,0.950457
3,0.080900,0.220323,0.949342,0.949375


TrainOutput(global_step=22500, training_loss=0.14848362189398873, metrics={'train_runtime': 1883.5102, 'train_samples_per_second': 191.132, 'train_steps_per_second': 11.946, 'total_flos': 2.368042020864e+16, 'train_loss': 0.14848362189398873, 'epoch': 3.0})

Step 8: Final Evaluation

In [19]:
results = trainer.evaluate()
print(results)


{'eval_loss': 0.17824463546276093, 'eval_accuracy': 0.9503947368421053, 'eval_f1': 0.9504574953167206, 'eval_runtime': 9.0741, 'eval_samples_per_second': 837.552, 'eval_steps_per_second': 52.347, 'epoch': 3.0}


Step 9: Save the Model

In [21]:
model.save_pretrained("news_classifier")
tokenizer.save_pretrained("news_classifier")

('news_classifier\\tokenizer_config.json',
 'news_classifier\\special_tokens_map.json',
 'news_classifier\\vocab.txt',
 'news_classifier\\added_tokens.json')

Step 10 : Deploy with Gradio (Live Demo)

In [23]:
import gradio as gr
import torch
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("news_classifier")
model = BertForSequenceClassification.from_pretrained("news_classifier")
model.eval()

labels = ["World", "Sports", "Business", "Sci/Tech"]

def classify_news(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return labels[prediction]

interface = gr.Interface(
    fn=classify_news,
    inputs=gr.Textbox(lines=2, placeholder="Enter a news headline..."),
    outputs="text",
    title="News Topic Classifier (BERT)",
    description="Classifies news headlines into World, Sports, Business, or Sci/Tech."
)

interface.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Final Summary / Insights:

The BERT-based news classification model achieved strong performance with an accuracy and F1-score of approximately 95%, demonstrating its effectiveness in capturing semantic context within news headlines. The results highlight the strength of transformer-based transfer learning for text classification tasks, even with minimal task-specific feature engineering. Overall, the model generalizes well across topic categories and is suitable for real-time deployment.